In [1]:
from turb.lesgo_utils import lesgo_data
import numpy as np
import turb.lesgo_utils
import matplotlib.pyplot as plt

# Generate IC files in input directory

In [ ]:
root_dir = '/home/zyou6474/tasks/test'

# Read data from root dir

In [2]:
root_dir = '/home/zyou6474/tasks/channel_flow'
dims = (128, 128, 64)
domain = (2*np.pi, np.pi, 1)

ldata = lesgo_data(domain, dims, root_dir, ntheta=3)

In [3]:
ldata.read_data(1)
ldata.data['theta'].shape

(3, 128, 128, 64)